In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing, utils
import re

import yaml
import os

In [8]:
# Load the saved model
new_model = tf.keras.models.load_model('test train with 5 files during 5 minutes')

# Show the model architecture
new_model.summary()

2023-06-22 23:11:26.594480: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-06-22 23:11:26.599164: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-06-22 23:11:26.606095: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-06-22 23:11:26.615451: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-06-22 23:11:26.707381: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond/while' has 13 outputs but the _ou

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 28)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 273)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 28, 200)              615400    ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, 273, 200)             615400    ['input_2[0][0]']             
                                                                                              

In [9]:
dir(new_model)

['_SCALAR_UPRANKING_ON',
 '_TF_MODULE_IGNORED_PROPERTIES',
 '__annotations__',
 '__call__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activity_regularizer',
 '_add_trackable',
 '_add_trackable_child',
 '_add_variable_with_custom_getter',
 '_aggregate_exact_metrics',
 '_assert_compile_was_called',
 '_assert_weights_created',
 '_auto_config',
 '_auto_get_config',
 '_auto_track_sub_layers',
 '_autocast',
 '_autographed_call',
 '_base_model_initialized',
 '_build_input_shape',
 '_call_spec',
 '_callable_losses',
 '_captured_weight_regularizer',
 '_cast_single_input',
 '_check_call_args',
 '_check_sampl

In [10]:
new_model.weights

[<tf.Variable 'embedding/embeddings:0' shape=(3077, 200) dtype=float32, numpy=
 array([[-0.06524513, -0.05904042, -0.05157605, ...,  0.04935744,
          0.02420027,  0.00851651],
        [-0.02488446,  0.02545274, -0.00546496, ...,  0.00404964,
          0.00843106,  0.04579191],
        [ 0.03603917, -0.0331744 ,  0.01575717, ...,  0.03185639,
          0.0372566 ,  0.03227321],
        ...,
        [-0.02484166, -0.01998086,  0.01637668, ..., -0.04628667,
          0.01729051,  0.03989622],
        [ 0.04088774,  0.03786981,  0.02447137, ..., -0.02960016,
         -0.02825652,  0.03864521],
        [ 0.00330367,  0.01173518,  0.01031369, ...,  0.01054559,
         -0.04416109,  0.04360357]], dtype=float32)>,
 <tf.Variable 'embedding_1/embeddings:0' shape=(3077, 200) dtype=float32, numpy=
 array([[ 0.04765953,  0.02517481, -0.04898639, ...,  0.02959828,
         -0.04162675,  0.03908109],
        [ 0.05571448,  0.01482712, -0.39123172, ..., -0.03442856,
         -0.10566137, -0.3706

In [2]:
dir_path = 'archive_test'
files_list = os.listdir(dir_path + os.sep)
questions, answers = [], []

for filepath in files_list:
    file_ = open(dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(file_)
    conversations = docs['conversations']
    for con in conversations:
        if len(con) > 2 :
            questions.append(con[0])
            replies = con[1 :]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append(ans)
        elif len(con)> 1:
            questions.append(con[0])
            answers.append(con[1])

answers_with_tags = []
for i in range(len(answers)):
    if type(answers[i]) == str:
        answers_with_tags.append(answers[i])
    else:
        questions.pop(i)

answers = []
for i in range(len(answers_with_tags)) :
    answers.append('<START> ' + answers_with_tags[i] + ' <END>')

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions + answers)
VOCAB_SIZE = len(tokenizer.word_index)+1

from gensim.models import Word2Vec
import re

vocab = []
for word in tokenizer.word_index:
    vocab.append(word)

def tokenize(sentences):
    tokens_list = []
    vocabulary = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub('[^a-zA-Z]', ' ', sentence)
        tokens = sentence.split()
        vocabulary += tokens
        tokens_list.append(tokens)
    return tokens_list , vocabulary

# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences(questions)
maxlen_questions = max([len(x) for x in tokenized_questions])
padded_questions = preprocessing.sequence.pad_sequences(tokenized_questions , maxlen=maxlen_questions , padding='post')
encoder_input_data = np.array(padded_questions)

encoder_input_data.shape

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences(answers)
maxlen_answers = max([len(x) for x in tokenized_answers])
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers , maxlen=maxlen_answers , padding='post')
decoder_input_data = np.array(padded_answers)

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences(answers)
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers , maxlen=maxlen_answers , padding='post')
onehot_answers = utils.to_categorical(padded_answers , VOCAB_SIZE)
decoder_output_data = np.array(onehot_answers)

# Embedding, LSTM and Desne layers
encoder_inputs = tf.keras.layers.Input(shape=(maxlen_questions ,))
encoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM(200 , return_state=True)(encoder_embedding)
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=(maxlen_answers , ))
decoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM(200 , return_state=True , return_sequences=True)
decoder_outputs , _ , _ = decoder_lstm (decoder_embedding , initial_state=encoder_states)


decoder_dense = tf.keras.layers.Dense(VOCAB_SIZE , activation=tf.keras.activations.softmax) 
output = decoder_dense (decoder_outputs)

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.load_weights('training_6files_300 epoch_30minutes')

2023-06-22 23:58:54.171498: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2023-06-22 23:58:54.171564: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-06-22 23:58:54.171567: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-06-22 23:58:54.171837: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-06-22 23:58:54.171857: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-06-22 23:58:54.775431: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open training_6files_300 epoch_30minutes: FAILED_PRECONDITION: training_6files_300 epoch_30minutes;

In [3]:
def inference():
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    decoder_state_input_h = tf.keras.layers.Input(shape=(200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=(200 ,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)
    return encoder_model , decoder_model
def preprocess_input(input_sentence):
    tokens = re.sub('[^a-zA-Z]', ' ', input_sentence).lower().split()
    # tokens = tokenize(input_sentence)
    print(tokens)
    tokens_list = []
    for word in tokens:
        tokens_list.append(tokenizer.word_index[word])
    return preprocessing.sequence.pad_sequences([tokens_list] , maxlen=maxlen_questions , padding='post')

In [4]:
enc_model , dec_model = inference()

In [5]:
tests = ['Hi?', 'In which parts of the learning model at Picsart Academy is JavaScript studied',
         'How many months last the course in Picsart Academy?', 'What is Sandbox at Picsart Academy', 'What courses there are in Level Up Bootcamp']

for i in range(5):
    states_values = enc_model.predict(preprocess_input(tests[i]))
    empty_target_seq = np.zeros((1 , 1))
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([empty_target_seq] + states_values)
        sampled_word_index = np.argmax(dec_outputs[0, -1, :])
        sampled_word = None
        
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += f' {word}'
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros((1 , 1))  
        empty_target_seq[0 , 0] = sampled_word_index
        states_values = [h , c] 
    print(f'Human: {tests[i]}')
    print()
    decoded_translation = decoded_translation.split(' end')[0]
    print(f'Bot: {decoded_translation}')
    print('-'*25)

['hi']


2023-06-22 23:59:28.525167: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-22 23:59:28.616157: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 13ms/step


2023-06-22 23:59:28.952881: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-22 23:59:29.040603: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 12ms/step
Human: Hi?

Bot:  the course will equip participants with valuable stress and conflict management skills that can positively impact their professional lives by effectively managing stress participants can improve their productivity decision making abilities and overall performance in the workplace the course will also focus on developing effective communication and interpersonal skills which are essential for building professional relationships and resolving conflicts in a constructive manner participants can expect to see enhanced professional growth increased job satisfaction and improved career prospects as a result of their improved stress and conflict management abilities
-------------------------
['in', 'which', 'parts', 'of', 'the', 'learning', 'model', 'at', 'picsart', 'academy', 'is', 'javascript', 'studied']
1/1 [==============================] - 0s 11ms/step
Human: In which parts of the learning model at Picsart Academy is 

1/1 [==============================] - 0s 11ms/step
Human: What is Sandbox at Picsart Academy

Bot:  the course will equip participants with valuable stress and conflict management skills that can positively impact their professional lives by effectively managing stress participants can improve their productivity decision making abilities and overall performance in the workplace the course will also focus on developing effective communication and interpersonal skills which are essential for building professional relationships and resolving conflicts in a constructive manner participants can expect to see enhanced professional growth increased job satisfaction and improved career prospects as a result of their improved stress and conflict management abilities
-------------------------
['what', 'courses', 'there', 'are', 'in', 'level', 'up', 'bootcamp']
1/1 [==============================] - 0s 11ms/step


1/1 [==============================] - 0s 11ms/step
Human: What courses there are in Level Up Bootcamp

Bot:  the course will equip participants with valuable stress and conflict management skills that can positively impact their professional lives by effectively managing stress participants can improve their productivity decision making abilities and overall performance in the workplace the course will also focus on developing effective communication and interpersonal skills which are essential for building professional relationships and resolving conflicts in a constructive manner participants can expect to see enhanced professional growth increased job satisfaction and improved career prospects as a result of their improved stress and conflict management abilities
-------------------------
